In [1]:
from BiasStudy import datasets, predictionKit

Python Platform: Linux-5.19.0-45-generic-x86_64-with-glibc2.35
Tensor Flow Version: 2.12.1
Keras Version: 2.12.0

Python 3.8.19 (default, Apr  6 2024, 17:58:10) 
[GCC 11.4.0]


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model


2024-04-28 20:37:27.216898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 20:37:28.316450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
BATCH_SIZE = 50

In [5]:
from BiasStudy.datasets import FairFaceDataset

fair_face_dataset = FairFaceDataset(
    data_dir = "/notebooks/data/fairface",
    train_labels_csv_name = "fairface_label_train.csv",
    validation_labels_csv_name = "fairface_label_val.csv",
    under_sample = True,
    image_shape = (224,224,3),
    feature_column = "file",
    output_col = "binary_race",
    overwrite_sample_number = 100
)

train_df = fair_face_dataset.get_train_pd()

Before Under Sampling:  {'light': 28814, 'dark': 24552}
After Under sampleing:  {'dark': 100, 'light': 100}


In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation="relu", input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(2))

model.compile(
    optimizer="rmsprop",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [10]:
model.get_layer("conv2d_2").trainable

False

In [16]:
len(model.trainable_variables)

8

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 173056)           

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [7]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size = (fair_face_dataset.get_image_width(),fair_face_dataset.get_image_height()),
    x_col = fair_face_dataset.get_feature_col_name(),
    y_col = fair_face_dataset.get_output_col_name(), 
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    subset = "training"
)

Found 160 validated image filenames belonging to 2 classes.


In [ ]:
type(train_generator)

In [8]:
validation_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size = (fair_face_dataset.get_image_width(),fair_face_dataset.get_image_height()),
    x_col = fair_face_dataset.get_feature_col_name(),
    y_col = fair_face_dataset.get_output_col_name(),
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    subset = "validation"
)

Found 40 validated image filenames belonging to 2 classes.


In [ ]:
history = []

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, EarlyStopping

In [ ]:
from pathlib import Path
Path("./outputs/checkpoints").mkdir(parents=True, exist_ok=True)
Path("./outputs/logging").mkdir(parents=True, exist_ok=True)
Path("./outputs/model").mkdir(parents=True, exist_ok=True)
Path("./outputs/weights").mkdir(parents=True, exist_ok=True)


In [ ]:
filepath="outputs/checkpoints/cp-{epoch:02d}.ckpt"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor = 'val_accuracy',
    verbose = 1
)

In [ ]:
log_csv = CSVLogger(
    filename = "outputs/logging/logs.csv",
    append = True
)

In [ ]:
callbacks_list = [checkpoint, log_csv]

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [ ]:
history.append(
    model.fit(
        train_generator,
        epochs = 2,
        validation_data = validation_generator,
        callbacks=callbacks_list
    )
)

In [ ]:
train_loss, train_acc = model.evaluate(train_generator)
validation_loss, test_acc = model.evaluate(validation_generator)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
model.save("outputs/model/model_name.h5")

In [ ]:
model.save_weights("./outputs/weights/model_weights")

In [2]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32,(3,3), activation="relu", input_shape=fair_face_dataset.get_image_shape()))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Conv2D(64, (3,3), activation="relu"))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Conv2D(64, (3,3), activation="relu"))
model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation="relu"))
model2.add(layers.Dense(2))

model2.compile(
    optimizer="rmsprop",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model2.load_weights("./outputs/weights/model_weights")

2024-04-28 20:37:34.069612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 20:37:34.117061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-28 20:37:34.117394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

NameError: name 'fair_face_dataset' is not defined

In [ ]:
train_loss, train_acc = model2.evaluate(train_generator)
validation_loss, test_acc = model2.evaluate(validation_generator)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
model3 = tf.keras.models.load_model("load_weights_name.h5")

In [ ]:
train_loss, train_acc = model3.evaluate(train_generator)
validation_loss, test_acc = model3.evaluate(validation_generator)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [15]:
import yaml

def load_config_and_validate(path: str) -> dict:
    with open(path, 'r') as stream:
        config = yaml.safe_load(stream)
    
    if "model" not in config:
        raise Exception("Missing field 'model'")
        
    if "model_name" not in config["model"]:
        raise Exception("Missing field 'model.model_name'")
        
    if "conv_blocks" not in config["model"]:
        raise Exception("Missing field 'model.conv_blocks'")
    
    return config

config_dict = load_config_and_validate("./sample.yaml")

In [ ]:
config_dict

In [9]:
from typing import Tuple

def create_model(
    num_classes: int,
    config_dict: dict,
    image_shape: Tuple[int, int, int] = (224,224,3)
):
    model_name = config_dict['model']['model_name']
    cnn_model = keras.Sequential(name=model_name)
    cnn_model.add(Input(shape=image_shape))
    input = Input(shape = image_shape)
    for block_num, (block_key, block_config) in enumerate(config_dict['model']['conv_blocks'].items()):
        for conv_layer_num in range(0, block_config['num_conv_layers']):
            cnn_model.add(
                Conv2D(
                    filters = block_config['num_filters'],
                    kernel_size = block_config['kernel_size'],
                    padding = 'same',
                    activation = 'relu',
                    name = "block{}_conv{}".format(block_num, conv_layer_num)
                )
            )
        cnn_model.add(MaxPool2D(pool_size =2, strides =2, padding ='same', name="block{}_pool".format(block_num)))
    cnn_model.add(Flatten(name = "flatten"))
    
    if "flatt_layers" in config_dict['model']:
        for flat_layer_num, (flat_key, flat_config) in enumerate(config_dict['model']['flatt_layers'].items()):
            cnn_model.add(Dense(units = flat_config['num_units'], activation ='relu'))

    cnn_model.add(Dense(units = num_classes, activation ='softmax', name = "prediction"))
    return cnn_model
                           

In [10]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

def compile_model(num_classes: int, cnn_model: keras.Model):
    loss = None
    if num_classes > 1:
        loss = "categorical_crossentropy"
    else:
        loss = "mean_squared_error"
    cnn_model.compile(
        optimizer = Adam(learning_rate=0.001),
        loss = loss,
        metrics = ['accuracy']
    )

In [11]:
# https://stackoverflow.com/a/72746245
class EpochModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):

    def __init__(self,
                 filepath,
                 frequency=1,
                 monitor='val_loss',
                 verbose=0,
                 save_best_only=False,
                 save_weights_only=False,
                 mode='auto',
                 options=None,
                 **kwargs):
        super(EpochModelCheckpoint, self).__init__(filepath, monitor, verbose, save_best_only, save_weights_only,
                                                   mode, "epoch", options)
        self.epochs_since_last_save = 0
        self.frequency = frequency

    def on_epoch_end(self, epoch, logs=None):
        self.epochs_since_last_save += 1
        # pylint: disable=protected-access
        if self.epochs_since_last_save % self.frequency == 0:
            self._save_model(epoch=epoch, batch=None, logs=logs)

    def on_train_batch_end(self, batch, logs=None):
        pass

In [12]:
from pathlib import Path

def create_dirs(model_name: str):
    checkpoint_dir = "./outputs/{}/checkpoints/".format(model_name)
    Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
    
    logging_dir = "./outputs/{}/logging/".format(model_name)
    Path(logging_dir).mkdir(parents=True, exist_ok=True)
    
    model_dir = "./outputs/{}/model/".format(model_name)
    Path(model_dir).mkdir(parents=True, exist_ok=True)
    
    weight_dir = "./outputs/{}/weights/".format(model_name)
    Path(weight_dir).mkdir(parents=True, exist_ok=True)
    
    return checkpoint_dir, logging_dir, model_dir, weight_dir

In [13]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras.preprocessing.image import DataFrameIterator

def train_model(
    early_stopping_patience: int,
    cnn_model: keras.Model,
    train_generator: DataFrameIterator,
    validation_generator: DataFrameIterator,
    num_epochs: int,
    checkpoint_frequencey: int = 1,
):
    checkpoint_dir, logging_dir, model_dir, weight_dir = create_dirs(cnn_model.name)
    
    checkpoint_file_path = checkpoint_dir + "cp-{epoch:02d}.ckpt"
    checkpoint_callback = EpochModelCheckpoint(
        filepath = checkpoint_file_path,
        monitor = 'val_accuracy',
        frequencey = checkpoint_frequencey,
        verbose = 1
    )
    
    log_csv_file_path = logging_dir + "logs.csv"
    log_csv_callback = CSVLogger(
        filename = log_csv_file_path,
        append = True
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=early_stopping_patience, 
        mode='min',
        min_delta=0.0001
    )

    callbacks_list = [checkpoint_callback, log_csv_callback, early_stopping]
    
    cnn_model.fit(
        train_generator,
        epochs = num_epochs,
        validation_data = validation_generator,
        callbacks = callbacks_list
    )
    
    train_loss, train_acc = cnn_model.evaluate(train_generator)
    validation_loss, test_acc = cnn_model.evaluate(validation_generator)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
    
    cnn_model.save(model_dir + "model.h5".format(cnn_model.name))
    cnn_model.save_weights(weight_dir)

In [16]:
cnn = create_model(num_classes = 2, config_dict = config_dict)
compile_model(num_classes = 2, cnn_model = cnn)
# cnn.summary()

In [17]:
train_model(
    cnn_model = cnn,
    train_generator = train_generator,
    validation_generator = validation_generator,
    num_epochs = 4,
    early_stopping_patience = 3,
    checkpoint_frequencey = 2
)

Epoch 1/4
4/4 [==============================] - ETA: 0s - loss: 2.2288 - accuracy: 0.6187
Epoch 1: saving model to ./outputs/model_8conv_3kernel/checkpoints/cp-01.ckpt


INFO:tensorflow:Assets written to: ./outputs/model_8conv_3kernel/checkpoints/cp-01.ckpt/assets


INFO:tensorflow:Assets written to: ./outputs/model_8conv_3kernel/checkpoints/cp-01.ckpt/assets


4/4 [==============================] - 16s 2s/step - loss: 2.2288 - accuracy: 0.6187 - val_loss: 0.3462 - val_accuracy: 1.0000
Epoch 2/4
4/4 [==============================] - ETA: 0s - loss: 0.9388 - accuracy: 0.5437
Epoch 2: saving model to ./outputs/model_8conv_3kernel/checkpoints/cp-02.ckpt


INFO:tensorflow:Assets written to: ./outputs/model_8conv_3kernel/checkpoints/cp-02.ckpt/assets


...

INFO:tensorflow:Assets written to: ./outputs/model_8conv_3kernel/checkpoints/cp-03.ckpt/assets


4/4 [==============================] - 2s 734ms/step - loss: 0.7116 - accuracy: 0.6375 - val_loss: 0.4889 - val_accuracy: 0.9500
Epoch 4/4
4/4 [==============================] - ETA: 0s - loss: 0.6299 - accuracy: 0.6250
Epoch 4: saving model to ./outputs/model_8conv_3kernel/checkpoints/cp-04.ckpt


INFO:tensorflow:Assets written to: ./outputs/model_8conv_3kernel/checkpoints/cp-04.ckpt/assets


INFO:tensorflow:Assets written to: ./outputs/model_8conv_3kernel/checkpoints/cp-04.ckpt/assets


1/1 [==============================] - 0s 162ms/step - loss: 0.6151 - accuracy: 0.8500
Train: 0.550, Test: 0.850


In [ ]:
# import pickle
# class History_trained_model(object):
#     def __init__(self, history, epoch, params):
#         self.history = history
#         self.epoch = epoch
#         self.params = params

# def save_history():
#     with open(savemodel_path+'/history', 'wb') as file:
#     model_history= History_trained_model(history.history, history.epoch, history.params)
#     pickle.dump(model_history, file, pickle.HIGHEST_PROTOCOL)

# def load_history():
#     with open(savemodel_path+'/history', 'rb') as file:
#         history=pickle.load(file)